### this noteboos save the metrics (static ones; and lss ones) for v1.4 dbs with shallower cuts.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import os
import pandas as pd
%matplotlib inline

In [2]:
data_dir = '/global/cscratch1/sd/awan/lsst_output/post_wp_output_v1.4_-0.1cuts/summary_data/'
outdir = '/global/homes/a/awan/LSST/lsstRepos/ObsStrat/postwp/paper-data/summary_csv_v1.4_-0.1cuts/'
os.makedirs(outdir, exist_ok=True)

#### read in the data for the static metrics and store as a dictionary

In [3]:
# read in the data 
data = {}
yr_label = {}
for yr in [1, 3, 6, 10]:
    files = [f for f in os.listdir( data_dir ) if f.endswith('csv') and f.__contains__('y%s_' % yr) and f.startswith('eg_') ]
    for file in files:
        print( file )
        key = 'yr%s_%s' % (yr, file.split('_')[4])
        data[ key ] = pd.read_csv('%s/%s' % (data_dir, file))
        yr_label[ key ] =  r'Y%s (i$>$%s) ' % (yr, file.split('_')[4].split('limi')[-1])
        
print( yr_label )
print(data.keys())
print(data[list(data.keys())[0]].keys() )

eg_footprint_stats_y1_limi24.65_nodither_nside256.csv
eg_footprint_stats_y3_limi25.25_nodither_nside256.csv
eg_footprint_stats_y6_limi25.619999999999997_nodither_nside256.csv
eg_footprint_stats_y10_limi25.9_nodither_nside256.csv
{'yr1_limi24.65': 'Y1 (i$>$24.65) ', 'yr3_limi25.25': 'Y3 (i$>$25.25) ', 'yr6_limi25.619999999999997': 'Y6 (i$>$25.619999999999997) ', 'yr10_limi25.9': 'Y10 (i$>$25.9) '}
dict_keys(['yr1_limi24.65', 'yr3_limi25.25', 'yr6_limi25.619999999999997', 'yr10_limi25.9'])
Index(['dbname', 'Area (deg2)', '$i$-band depth: median',
       ' $i$-band depth: std'],
      dtype='object')


In [4]:
# read in the static metrics
for key in data:
    yr_tag = key.split('_')[0].split('yr')[-1]
    lim_tag = key.split('_')[1].split('limi')[-1]
    print(yr_tag, lim_tag)

    data[key] = data[key].rename(columns={'Area (deg2)': 'Y%s effective survey area ' % (yr_tag),
                                          '$i$-band depth: median': 'median Y%s $i$-band coadded depth in effective survey area' % (yr_tag),
                                          ' $i$-band depth: std': 'std in Y%s $i$-band coadded depth in effective survey area' % (yr_tag)} )
    #fname = 'lss_metrics_%s.csv' % key
    #data[key].to_csv('%s/%s' % (outdir, fname), index=False)

1 24.65
3 25.25
6 25.619999999999997
10 25.9


#### read in the `list.csv` file: gives us the order of the dbs

In [5]:
given_list = pd.read_csv('%s/order_list.csv' % outdir, )
given_list

,name
0,agnddf_v1.4_10yrs.db
1,alt_roll_mod2_dust_sdf_0.20_v1.4_10yrs.db
2,baseline_2snapsv1.4_10yrs.db
3,baseline_v1.4_10yrs.db
4,bulges_bs_v1.4_10yrs.db
...,...
70,wfd_depth_scale0.90_v1.4_10yrs.db
71,wfd_depth_scale0.95_noddf_v1.4_10yrs.db
72,wfd_depth_scale0.95_v1.4_10yrs.db
73,wfd_depth_scale0.99_noddf_v1.4_10yrs.db


In [6]:
# check some things:
len( list( given_list['name'] ) ), len( np.unique( given_list['name'] ) )

(75, 75)

In [7]:
yr_key = [f for f in data.keys() if f.__contains__('yr1_')][0]
yr_key

'yr1_limi24.65'

#### find the indices to get the order

In [8]:
# find the indices to get the order
ind = []
for db in given_list['name']:
    ind_temp = np.where( data[yr_key]['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

[40  6 15 63 25 53 30 43 20  4 51 36 45 27 66 58 18 72 19 33 12  1 54 60
 32  7 26 67  2 28 65 71 44 38 39 37 13 46 57 34  0  9 23 24 56 16 49  5
 69 74 42 29 14  8 52 64 48 68 31 62 35 21 61 11 50 10 59 70 73 41 55  3
 47 22 17]


In [9]:
# check we have the right indices
print( data[yr_key]['dbname'].values[40],  given_list['name'][0] )

# check we have the right number of inds
print( len(ind), len(np.unique(ind)) )

agnddf_v1.4_10yrs agnddf_v1.4_10yrs.db
75 75


#### save the data in the order in which to save things

In [10]:
for yr_lim_tag in data:
    for colname in data[yr_lim_tag]:
        if colname != 'dbname':
             given_list[colname] = data[yr_lim_tag][colname].values[ind]

In [11]:
given_list

,name,Y1 effective survey area,median Y1 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area,Y3 effective survey area,median Y3 $i$-band coadded depth in effective survey area,std in Y3 $i$-band coadded depth in effective survey area,Y6 effective survey area,median Y6 $i$-band coadded depth in effective survey area,std in Y6 $i$-band coadded depth in effective survey area,Y10 effective survey area,median Y10 $i$-band coadded depth in effective survey area,std in Y10 $i$-band coadded depth in effective survey area
0,agnddf_v1.4_10yrs.db,15164.88,25.10,0.16,15268.85,25.71,0.14,15201.44,26.05,0.14,15154.65,26.31,0.13
1,alt_roll_mod2_dust_sdf_0.20_v1.4_10yrs.db,16734.73,25.00,0.16,16378.82,25.54,0.18,15909.07,25.92,0.15,16027.99,26.19,0.13
2,baseline_2snapsv1.4_10yrs.db,14943.10,25.04,0.15,15148.78,25.65,0.13,14986.85,25.99,0.13,14926.47,26.25,0.12
3,baseline_v1.4_10yrs.db,15124.18,25.09,0.16,15250.44,25.69,0.14,15107.50,26.03,0.14,15048.22,26.29,0.13
4,bulges_bs_v1.4_10yrs.db,16865.19,24.99,0.16,17205.94,25.62,0.14,16903.37,25.96,0.14,16717.84,26.22,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,wfd_depth_scale0.90_v1.4_10yrs.db,15143.95,25.08,0.16,15239.74,25.69,0.13,15140.02,26.02,0.13,15086.78,26.29,0.13
71,wfd_depth_scale0.95_noddf_v1.4_10yrs.db,15268.06,25.17,0.17,15327.76,25.78,0.15,15263.18,26.12,0.14,15242.99,26.38,0.14
72,wfd_depth_scale0.95_v1.4_10yrs.db,15116.05,25.09,0.15,15237.32,25.71,0.14,15154.81,26.05,0.13,15131.31,26.31,0.13
73,wfd_depth_scale0.99_noddf_v1.4_10yrs.db,15254.06,25.18,0.17,15304.41,25.79,0.15,15258.15,26.14,0.14,15238.95,26.40,0.14


In [12]:
# save the metrics
fname = 'static_metrics.csv'
given_list.to_csv('%s/%s' % (outdir, fname), index=False)

#### now read and save lss-fom and ngal

In [13]:
# clean up the give_list dataframe
for key in given_list.keys():
    if key.__contains__('Y'):
        given_list = given_list.drop(key, axis=1)
        
given_list

,name
0,agnddf_v1.4_10yrs.db
1,alt_roll_mod2_dust_sdf_0.20_v1.4_10yrs.db
2,baseline_2snapsv1.4_10yrs.db
3,baseline_v1.4_10yrs.db
4,bulges_bs_v1.4_10yrs.db
...,...
70,wfd_depth_scale0.90_v1.4_10yrs.db
71,wfd_depth_scale0.95_noddf_v1.4_10yrs.db
72,wfd_depth_scale0.95_v1.4_10yrs.db
73,wfd_depth_scale0.99_noddf_v1.4_10yrs.db


#### read in the lss fom values

In [14]:
data_dir = '/global/cscratch1/sd/awan/lsst_output/post_wp_output_v1.4_-0.1cuts/lss_fom/os_bias_overplots/'
[f for f in os.listdir(data_dir) if f.__contains__('fomdata')] 

['2020-03-06_fomdata_Y1_i<23.95_snfom_egfootprint_75cadences_0.66<z<1.0_th-r<24.35_100<ell<300_logtcuts.csv',
 '2020-03-06_fomdata_Y10_i<25.2_snfom_egfootprint_75cadences_0.66<z<1.0_th-r<25.60_100<ell<300_logtcuts.csv']

In [15]:
data_file = '%s/2020-03-06_fomdata_Y1_i<23.95_snfom_egfootprint_75cadences_0.66<z<1.0_th-r<24.35_100<ell<300_logtcuts.csv' % data_dir
lss_fom = pd.read_csv(data_file)

# find the indices to get the order
ind = []
for db in given_list['name']:
    ind_temp = np.where( lss_fom['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

given_list['Y1 lss fom'] = lss_fom['FoM'].values[ind]

[40  6 15 63 25 53 30 43 20  4 51 36 45 27 66 58 18 72 19 33 12  1 54 60
 32  7 26 67  2 28 65 71 44 38 39 37 13 46 57 34  0  9 23 24 56 16 49  5
 69 74 42 29 14  8 52 64 48 68 31 62 35 21 61 11 50 10 59 70 73 41 55  3
 47 22 17]


In [16]:
data_file = '%s/2020-03-06_fomdata_Y10_i<25.2_snfom_egfootprint_75cadences_0.66<z<1.0_th-r<25.60_100<ell<300_logtcuts.csv' % data_dir
lss_fom = pd.read_csv(data_file)

# find the indices to get the order
ind = []
for db in given_list['name']:
    ind_temp = np.where( lss_fom['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

given_list['Y10 lss fom'] = lss_fom['FoM'].values[ind]

[40  6 15 63 25 53 30 43 20  4 51 36 45 27 66 58 18 72 19 33 12  1 54 60
 32  7 26 67  2 28 65 71 44 38 39 37 13 46 57 34  0  9 23 24 56 16 49  5
 69 74 42 29 14  8 52 64 48 68 31 62 35 21 61 11 50 10 59 70 73 41 55  3
 47 22 17]


In [17]:
given_list

,name,Y1 lss fom,Y10 lss fom
0,agnddf_v1.4_10yrs.db,0.897023,0.988911
1,alt_roll_mod2_dust_sdf_0.20_v1.4_10yrs.db,0.892612,0.975412
2,baseline_2snapsv1.4_10yrs.db,0.894316,0.972489
3,baseline_v1.4_10yrs.db,0.906006,0.980298
4,bulges_bs_v1.4_10yrs.db,0.895837,1.009050
...,...,...,...
70,wfd_depth_scale0.90_v1.4_10yrs.db,0.893714,0.983406
71,wfd_depth_scale0.95_noddf_v1.4_10yrs.db,0.905526,0.997895
72,wfd_depth_scale0.95_v1.4_10yrs.db,0.903359,0.988475
73,wfd_depth_scale0.99_noddf_v1.4_10yrs.db,0.909466,0.999343


#### now read in the ngal values

In [18]:
ngal_data_dir = '/global/cscratch1/sd/awan/lsst_output/post_wp_output_v1.4_-0.1cuts/lss_fom/'

ngal_data = {}
for yr in [1, 10]:
    ngal_data[yr] =  np.zeros(len(given_list.name))
    folders = [f for f in os.listdir( ngal_data_dir ) if f.__contains__('_Y%s_' % yr) and f.__contains__('i<') ]
    for i, folder in enumerate( folders ):
        out = pd.read_csv('%s/%s/ngal-final_total/ngal_NoDither.txt' % (ngal_data_dir, folder), header=None, delimiter=' ') 
        dbname, ngal = out[0].values[0], out[1].values[0]
        ind = np.where( given_list['name'].values == '%s.db' % dbname)[0]
        ngal_data[yr][ind] = ngal
    print('read in %s folders for Y%s' % (i+1, yr) )

read in 75 folders for Y1
read in 75 folders for Y10


In [19]:
for yr in ngal_data:
    given_list['Y%s Ngal (0.66<z<1.0)' % yr] = ngal_data[yr]

In [20]:
given_list

,name,Y1 lss fom,Y10 lss fom,Y1 Ngal (0.66<z<1.0),Y10 Ngal (0.66<z<1.0)
0,agnddf_v1.4_10yrs.db,0.897023,0.988911,431000000.0,919000000.0
1,alt_roll_mod2_dust_sdf_0.20_v1.4_10yrs.db,0.892612,0.975412,448000000.0,888000000.0
2,baseline_2snapsv1.4_10yrs.db,0.894316,0.972489,416000000.0,885000000.0
3,baseline_v1.4_10yrs.db,0.906006,0.980298,429000000.0,901000000.0
4,bulges_bs_v1.4_10yrs.db,0.895837,1.009050,450000000.0,955000000.0
...,...,...,...,...,...
70,wfd_depth_scale0.90_v1.4_10yrs.db,0.893714,0.983406,429000000.0,908000000.0
71,wfd_depth_scale0.95_noddf_v1.4_10yrs.db,0.905526,0.997895,439000000.0,939000000.0
72,wfd_depth_scale0.95_v1.4_10yrs.db,0.903359,0.988475,430000000.0,919000000.0
73,wfd_depth_scale0.99_noddf_v1.4_10yrs.db,0.909466,0.999343,439000000.0,942000000.0


In [21]:
fname = 'lss_metrics.csv'
given_list.to_csv('%s/%s' % (outdir, fname), index=False)